In [2]:
import os
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

%matplotlib inline

# import cv2
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook #, tnrange
#from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import optimizers

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img#,save_img

Using TensorFlow backend.


In [17]:
def BatchActivate(x,act):
    x = BatchNormalization()(x)
    x = Activation(act)(x)
    return x


def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True,act='elu'):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x,act)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False,act='elu'):
    x = BatchActivate(blockInput,act)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x,act)
    return x

In [127]:
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    # 101 -> 50
    print("££££££££££££££££££££££££££££ Stage 1 £££££££££££££££££££££££££££££££££££££££££££££")
    print("input shape: {}".format(input_layer.shape))
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    print("first conv after input : {}".format(conv1.shape))
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    print("shape after two conv {}".format(conv1.shape))
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)
    print("shape after maxpool and dropout {}".format(pool1.shape))
    
    print("££££££££££££££££££££££££££££ Stage 2 £££££££££££££££££££££££££££££££££££££££££££££")
    # 50 -> 25
    print("input shape: {}".format(pool1.shape))
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    print("first conv after input : {}".format(conv2.shape))
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    print("shape after two conv {}".format(conv2.shape))
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)
    print("shape after maxpool and dropout {}".format(pool2.shape))
    
    print("££££££££££££££££££££££££££££ Stage 3 £££££££££££££££££££££££££££££££££££££££££££££")
    # 25 -> 12
    print("input shape: {}".format(pool2.shape))
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    print("first conv after input : {}".format(conv3.shape))
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    print("shape after two conv {}".format(conv3.shape))
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)
    print("shape after maxpool and dropout {}".format(pool3.shape))
    
    print("££££££££££££££££££££££££££££ Stage 4 £££££££££££££££££££££££££££££££££££££££££££££")
    # 12 -> 6
    print("input shape: {}".format(pool3.shape))
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    print("first conv after input : {}".format(conv4.shape))
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8, True)
    print("shape after two conv {}".format(conv4.shape))
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)
    print("shape after maxpool and dropout {}".format(pool4.shape))

    print("££££££££££££££££££££££££££££ Stage middle £££££££££££££££££££££££££££££££££££££££££££££")
    # Middle
    print("input shape: {}".format(pool4.shape))
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    print("first conv after input : {}".format(convm.shape))
    convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16, True)
    print("shape after two conv and this is last shape middle {}".format(convm.shape))
    
    print("££££££££££££££££££££££££££££ Stage 4 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 6 -> 12
    print("input shape convm: {}".format(convm.shape))
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    print("first deconv after transpose convm -> deconv4: {}".format(deconv4.shape))
    print("conv 4 shape for concat : {}".format(conv4.shape))
    uconv4 = concatenate([deconv4, conv4])
    print("shape after concat deconv4, conv4 {}".format(uconv4.shape))
    uconv4 = Dropout(DropoutRatio)(uconv4)
    print("shape after dropout {}".format(uconv4.shape))
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    print("conv2d not activation: {}".format(uconv4.shape))
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8, True)
    print("shape after two conv and las shape {}".format(uconv4.shape))
    
    
    print("££££££££££££££££££££££££££££ Stage 3 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 12 -> 25
    print("input shape: {}".format(uconv4.shape))
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    print("first deconv after transpose deconv3 : {}".format(deconv3.shape))
    print("conv 3 shape for concat : {}".format(conv3.shape))
    uconv3 = concatenate([deconv3, conv3]) 
    print("shape after concat deconv4, conv4 {}".format(uconv3.shape))
    uconv3 = Dropout(DropoutRatio)(uconv3)
    print("shape after dropout {}".format(uconv3.shape))
    
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    print("conv2d not activation: {}".format(uconv3.shape))
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4, True)
    print("shape after two conv and las shape {}".format(uconv3.shape))
    
    
    print("££££££££££££££££££££££££££££ Stage 2 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 25 -> 50
    print("input shape: {}".format(uconv3.shape))
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    print("first deconv after transpose deconv2 : {}".format(deconv2.shape))
    print("conv 2 shape for concat : {}".format(conv2.shape))
    uconv2 = concatenate([deconv2, conv2])
    print("shape after concat deconv4, conv4 {}".format(uconv2.shape))
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    print("shape after dropout {}".format(uconv2.shape))
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    print("conv2d not activation: {}".format(uconv2.shape))
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2, True)
    print("shape after two conv and las shape {}".format(uconv2.shape))
    
    print("££££££££££££££££££££££££££££ Stage 1 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 50 -> 101
    print("input shape: {}".format(uconv2.shape))
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    print("first deconv after transpose deconv1 : {}".format(deconv1.shape))
    print("conv 1 shape for concat : {}".format(conv1.shape))
    uconv1 = concatenate([deconv1, conv1])
    print("shape after concat deconv4, conv4 {}".format(uconv1.shape))
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    print("shape after dropout {}".format(uconv1.shape))
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    print("conv2d not activation: {}".format(uconv1.shape))
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    print("shape after two conv and las shape {}".format(uconv1.shape))
    
    print("££££££££££££££££££££££££££££ last stage for sigmoid £££££££££££££££££££££££££££££££££££££££££££££")
    print("input shape: {}".format(uconv1.shape))
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)(uconv1)
    print("first conv after input : {}".format(output_layer_noActi.shape))
    output_layer =  Activation('sigmoid')(output_layer_noActi)
    print("shape after two Activation and las shape {}".format(output_layer.shape))
    
    return output_layer

In [128]:
input_layer = Input((101, 101, 1),name="input")
output_layer = build_model(input_layer, 16,0.5)

££££££££££££££££££££££££££££ Stage 1 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 101, 101, 1)
first conv after input : (?, 101, 101, 16)
shape after two conv (?, 101, 101, 16)
shape after maxpool and dropout (?, 50, 50, 16)
££££££££££££££££££££££££££££ Stage 2 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 50, 50, 16)
first conv after input : (?, 50, 50, 32)
shape after two conv (?, 50, 50, 32)
shape after maxpool and dropout (?, 25, 25, 32)
££££££££££££££££££££££££££££ Stage 3 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 25, 25, 32)
first conv after input : (?, 25, 25, 64)
shape after two conv (?, 25, 25, 64)
shape after maxpool and dropout (?, 12, 12, 64)
££££££££££££££££££££££££££££ Stage 4 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 12, 12, 64)
first conv after input : (?, 12, 12, 128)
shape after two conv (?, 12, 12, 128)
shape after maxpool and dropout (?, 6, 6, 128)
££££££££££££££££££££££££££££ Stage middle 

In [42]:
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    # 101 -> 50
    print("££££££££££££££££££££££££££££ Stage 1 £££££££££££££££££££££££££££££££££££££££££££££")
    print("input shape: {}".format(input_layer.shape))
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    print("first conv after input : {}".format(conv1.shape))
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    print("shape after two conv {}".format(conv1.shape))
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)
    print("shape after maxpool and dropout {}".format(pool1.shape))
    
    print("££££££££££££££££££££££££££££ Stage 2 £££££££££££££££££££££££££££££££££££££££££££££")
    # 50 -> 25
    print("input shape: {}".format(pool1.shape))
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    print("first conv after input : {}".format(conv2.shape))
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    print("shape after two conv {}".format(conv2.shape))
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)
    print("shape after maxpool and dropout {}".format(pool2.shape))
    
    print("££££££££££££££££££££££££££££ Stage 3 £££££££££££££££££££££££££££££££££££££££££££££")
    # 25 -> 12
    print("input shape: {}".format(pool2.shape))
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    print("first conv after input : {}".format(conv3.shape))
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    print("shape after two conv {}".format(conv3.shape))
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)
    print("shape after maxpool and dropout {}".format(pool3.shape))
    
    print("££££££££££££££££££££££££££££ Stage 4 £££££££££££££££££££££££££££££££££££££££££££££")
    # 12 -> 6
    print("input shape: {}".format(pool3.shape))
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    print("first conv after input : {}".format(conv4.shape))
    conv4 = residual_block(conv4,start_neurons * 8,act="relu")
    conv4 = residual_block(conv4,start_neurons * 8, True,act="relu")
    print("shape after two conv {}".format(conv4.shape))
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)
    print("shape after maxpool and dropout {}".format(pool4.shape))

    print("££££££££££££££££££££££££££££ Stage middle £££££££££££££££££££££££££££££££££££££££££££££")
    # Middle
    print("input shape: {}".format(pool4.shape))
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    print("first conv after input : {}".format(convm.shape))
    convm = residual_block(convm,start_neurons * 16,act="relu")
    convm = residual_block(convm,start_neurons * 16, True,act="relu")
    print("shape after two conv and this is last shape middle {}".format(convm.shape))
    
    print("££££££££££££££££££££££££££££ Stage 4 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 6 -> 12
    print("input shape convm: {}".format(convm.shape))
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    print("first deconv after transpose convm -> deconv4: {}".format(deconv4.shape))
    print("conv 4 shape for concat : {}".format(conv4.shape))
    uconv4 = concatenate([deconv4, conv4])
    print("shape after concat deconv4, conv4 {}".format(uconv4.shape))
    uconv4 = Dropout(DropoutRatio)(uconv4)
    print("shape after dropout {}".format(uconv4.shape))
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    print("conv2d not activation: {}".format(uconv4.shape))
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8, True)
    print("shape after two conv and las shape {}".format(uconv4.shape))
    
    
    print("££££££££££££££££££££££££££££ Stage 3 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 12 -> 25
    print("input shape: {}".format(uconv4.shape))
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    print("first deconv after transpose deconv3 : {}".format(deconv3.shape))
    print("conv 3 shape for concat : {}".format(conv3.shape))
    uconv3 = concatenate([deconv3, conv3]) 
    print("shape after concat deconv4, conv4 {}".format(uconv3.shape))
    uconv3 = Dropout(DropoutRatio)(uconv3)
    print("shape after dropout {}".format(uconv3.shape))
    
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    print("conv2d not activation: {}".format(uconv3.shape))
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4, True)
    print("shape after two conv and las shape {}".format(uconv3.shape))
    
    
    print("££££££££££££££££££££££££££££ Stage 2 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 25 -> 50
    print("input shape: {}".format(uconv3.shape))
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    print("first deconv after transpose deconv2 : {}".format(deconv2.shape))
    print("conv 2 shape for concat : {}".format(conv2.shape))
    uconv2 = concatenate([deconv2, conv2])
    print("shape after concat deconv4, conv4 {}".format(uconv2.shape))
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    print("shape after dropout {}".format(uconv2.shape))
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    print("conv2d not activation: {}".format(uconv2.shape))
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2, True)
    print("shape after two conv and las shape {}".format(uconv2.shape))
    
    print("££££££££££££££££££££££££££££ Stage 1 up £££££££££££££££££££££££££££££££££££££££££££££")
    # 50 -> 101
    print("input shape: {}".format(uconv2.shape))
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    print("first deconv after transpose deconv1 : {}".format(deconv1.shape))
    print("conv 1 shape for concat : {}".format(conv1.shape))
    uconv1 = concatenate([deconv1, conv1])
    print("shape after concat deconv4, conv4 {}".format(uconv1.shape))
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    print("shape after dropout {}".format(uconv1.shape))
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    print("conv2d not activation: {}".format(uconv1.shape))
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    print("shape after two conv and las shape {}".format(uconv1.shape))
    
    print("££££££££££££££££££££££££££££ last stage for unet 1 £££££££££££££££££££££££££££££££££££££££££££££")
#     print("input shape: {}".format(uconv1.shape))
#     #uconv1 = Dropout(DropoutRatio/2)(uconv1)
#     #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
#     output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)(uconv1)
#     print("first conv after input : {}".format(output_layer_noActi.shape))
#     output_layer =  Activation('sigmoid')(output_layer_noActi)
#     print("shape after two Activation and las shape {}".format(output_layer.shape))

#     u-net 2 
    print("££££££££££££££££££££££££££££ Stage 5 £££££££££££££££££££££££££££££££££££££££££££££")
    print("££££££££££££££££££££££££££££ Stage 5 £££££££££££££££££££££££££££££££££££££££££££££")
    print("££££££££££££££££££££££££££££ Stage 5 £££££££££££££££££££££££££££££££££££££££££££££")
    # 101 -> 50
    print("input shape: {}".format(input_layer.shape))
    print("input from previous unet :{}".format(uconv1.shape))
    conv5 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    print("first conv after input : {}".format(conv5.shape))
    conv5 = concatenate([conv5, uconv1])
    print("shape after concat {}".format(conv5.shape))
    conv5 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(conv5)
    print("shape after concat and back to 16 {}".format(conv5.shape))
#     conv5 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same",name="new_unet")(uconv1)
    conv5 = residual_block(conv5,start_neurons * 1)
    conv5 = residual_block(conv5,start_neurons * 1, True)
    pool5 = MaxPooling2D((2, 2))(conv5)
    pool5 = Dropout(DropoutRatio/2)(pool5)
    print("after pool {}".format(pool5))
    
    print("££££££££££££££££££££££££££££ Stage 6 £££££££££££££££££££££££££££££££££££££££££££££")
    # 50 -> 25
    print("input shape: {}".format(pool5.shape))
    print("input from previous unet :{}".format(uconv2.shape))
    conv6 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool5)
    print("shape pool5 to conv6 {}".format(conv6.shape))
    conv6 = concatenate([conv6, uconv2])
    print("shape after concat {}".format(conv6.shape))
    conv6 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(conv6)
    print("shape concat and back to normal {}".format(conv6.shape))
    conv6 = residual_block(conv6,start_neurons * 2)
    conv6 = residual_block(conv6,start_neurons * 2, True)
    print("shape after conv 2 {}".format(conv6.shape))
    pool6 = MaxPooling2D((2, 2))(conv6)
    pool6 = Dropout(DropoutRatio)(pool6)
    print("shape after max and drop {}".format(pool6.shape))
    
    print("££££££££££££££££££££££££££££ Stage 7 £££££££££££££££££££££££££££££££££££££££££££££")
    # 25 -> 12
    print("input shape: {}".format(pool6.shape))
    print("input from previous unet :{}".format(uconv3.shape))
    conv7 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool6)
    print("shape pool6 to conv7 {}".format(conv7.shape))
    conv7 = concatenate([conv7, uconv3])
    print("shape after concat {}".format(conv7.shape))
    conv7 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(conv7)
    print("shape concat and back to normal {}".format(conv7.shape))
    conv7 = residual_block(conv7,start_neurons * 4)
    conv7 = residual_block(conv7,start_neurons * 4, True)
    print("shape after conv 2 {}".format(conv7.shape))
    pool7 = MaxPooling2D((2, 2))(conv7)
    pool7 = Dropout(DropoutRatio)(pool7)
    print("shape after max and drop {}".format(pool7.shape))
    
    print("££££££££££££££££££££££££££££ Stage 8 £££££££££££££££££££££££££££££££££££££££££££££")
    # 12 -> 6
    print("input shape: {}".format(pool7.shape))
    print("input from previous unet :{}".format(uconv4.shape))
    conv8 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool7)
    print("shape pool7 to conv8 {}".format(conv8.shape))
    conv8 = concatenate([conv8, uconv4])
    print("shape after concat {}".format(conv8.shape))
    conv8 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(conv8)
    print("shape concat and back to normal {}".format(conv8.shape))
    conv8 = residual_block(conv8,start_neurons * 8,act='relu')
    conv8 = residual_block(conv8,start_neurons * 8, True,act='relu')
    print("shape after conv 2 {}".format(conv8.shape))
    pool8 = MaxPooling2D((2, 2))(conv8)
    pool8 = Dropout(DropoutRatio)(pool8)
    print("shape after max and drop {}".format(pool8.shape))
    
    print("££££££££££££££££££££££££££££ Stage unet 2 middle £££££££££££££££££££££££££££££££££££££££££££££")
    # Middle
    print("input shape: {}".format(pool8.shape))
    print("input from previous unet :{}".format(convm.shape))
    convm_ = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool8)
    print("shape pool8 to convm_ {}".format(convm_.shape))
    convm_ = concatenate([convm_, convm])
    print("shape after concat {}".format(convm_.shape))
    convm_ = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(concatenate([pool8,convm]))
    print("shape concat and back to normal {}".format(convm_.shape))
    convm_ = residual_block(convm_,start_neurons * 16,act='relu')
    convm_ = residual_block(convm_,start_neurons * 16, True,act='relu')
    print("shape after conv 2 {}".format(convm_.shape))
    # 6 -> 12
    print("££££££££££££££££££££££££££££ Stage 8 up £££££££££££££££££££££££££££££££££££££££££££££")
    print("input shape: {}".format(convm_.shape))
    deconv8 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm_)
    print("transpose shape: {}".format(deconv8.shape))
    print("shape conv8 {}".format(conv8.shape))
    print("shape conv4 {}".format(conv4.shape))
    print("shape skip connection conv4 conv 8 {}".format(add([conv8, conv4]).shape))
    uconv8 = concatenate([deconv8, add([conv8, conv4])])
    uconv8 = Dropout(DropoutRatio)(uconv8)
    print("shape after droput {}".format(uconv8.shape))
    uconv8 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv8)
    print("conv2d {}".format(uconv8.shape))
    uconv8 = residual_block(uconv8,start_neurons * 8)
    uconv8 = residual_block(uconv8,start_neurons * 8, True)
    print(" last shpae after 2 conv {}".format(uconv8.shape))
    
    # 12 -> 25
    print("££££££££££££££££££££££££££££ Stage 7 up £££££££££££££££££££££££££££££££££££££££££££££")
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv7 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv8)
    uconv7 = concatenate([deconv7, add([conv7,conv3]) ])    
    uconv7 = Dropout(DropoutRatio)(uconv7)
    
    uconv7 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv7)
    uconv7 = residual_block(uconv7,start_neurons * 4)
    uconv7 = residual_block(uconv7,start_neurons * 4, True)

    # 25 -> 50
    print("££££££££££££££££££££££££££££ Stage 6 up £££££££££££££££££££££££££££££££££££££££££££££")
    deconv6 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv7)
    uconv6 = concatenate([deconv6, add([conv6,conv2])])
        
    uconv6 = Dropout(DropoutRatio)(uconv6)
    uconv6 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv6)
    uconv6 = residual_block(uconv6,start_neurons * 2)
    uconv6 = residual_block(uconv6,start_neurons * 2, True)
    
    # 50 -> 101
    print("££££££££££££££££££££££££££££ Stage 5 up £££££££££££££££££££££££££££££££££££££££££££££")
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv5 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv6)
    uconv5 = concatenate([deconv5, add([conv5,conv1])])
    
    uconv5 = Dropout(DropoutRatio)(uconv5)
    uconv5 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv5)
    uconv5 = residual_block(uconv5,start_neurons * 1)
    uconv5 = residual_block(uconv5,start_neurons * 1, True)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
# ////////////////////////////////////////////////////////////////////////////
    
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)( uconv5)
    output_layer =  Activation('sigmoid')(output_layer_noActi)
    
    return output_layer

In [43]:
input_layer = Input((101, 101, 1),name="input")
output_layer = build_model(input_layer, 16,0.5)

££££££££££££££££££££££££££££ Stage 1 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 101, 101, 1)
first conv after input : (?, 101, 101, 16)
shape after two conv (?, 101, 101, 16)
shape after maxpool and dropout (?, 50, 50, 16)
££££££££££££££££££££££££££££ Stage 2 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 50, 50, 16)
first conv after input : (?, 50, 50, 32)
shape after two conv (?, 50, 50, 32)
shape after maxpool and dropout (?, 25, 25, 32)
££££££££££££££££££££££££££££ Stage 3 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 25, 25, 32)
first conv after input : (?, 25, 25, 64)
shape after two conv (?, 25, 25, 64)
shape after maxpool and dropout (?, 12, 12, 64)
££££££££££££££££££££££££££££ Stage 4 £££££££££££££££££££££££££££££££££££££££££££££
input shape: (?, 12, 12, 64)
first conv after input : (?, 12, 12, 128)
shape after two conv (?, 12, 12, 128)
shape after maxpool and dropout (?, 6, 6, 128)
££££££££££££££££££££££££££££ Stage middle 

In [132]:

model1 = Model(input_layer, output_layer)

In [109]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 101, 101, 1)       0         
_________________________________________________________________
input_afterconv2d (Conv2D)   (None, 101, 101, 16)      160       
_________________________________________________________________
batch1 (BatchNormalization)  (None, 101, 101, 16)      64        
_________________________________________________________________
activation1 (Activation)     (None, 101, 101, 16)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 101, 101, 16)      2320      
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 50, 50, 32)        4640      
__________

In [7]:
from keras.utils import plot_model
plot_model(model1)

In [10]:
first_layer = Input((101, 101, 1),name="first")
second_layer = Input((101, 101, 16),name="second")
addition = add([first_layer,second_layer])

In [5]:
addition.shape

TensorShape([Dimension(None), Dimension(101), Dimension(101), Dimension(1)])

In [6]:
concat = concatenate([first_layer,second_layer])

In [7]:
concat.shape

TensorShape([Dimension(None), Dimension(101), Dimension(101), Dimension(2)])

In [13]:
transpose = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding="valid")(first_layer)

In [14]:
transpose.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(16)])